<a href="https://colab.research.google.com/github/thanugeorge13/BIS_LAB/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random

# Parameters
CHROM_LENGTH = 5     # binary string length (0–31)
POP_SIZE = 4         # number of chromosomes
CROSS_RATE = 0.8
MUT_RATE = 0.1

# Fitness function
def fitness(x):
    return x**2

# Convert decimal to binary string
def encode(x):
    return format(x, f'0{CHROM_LENGTH}b')

# Convert binary string to decimal
def decode(b):
    return int(b, 2)

# Roulette wheel selection
def roulette_selection(pop, fitnesses):
    total_fit = sum(fitnesses)
    pick = random.uniform(0, total_fit)
    current = 0
    for i, f in enumerate(fitnesses):
        current += f
        if current > pick:
            return pop[i]
    return pop[-1]

# Single-point crossover
def crossover(p1, p2):
    if random.random() < CROSS_RATE:
        point = random.randint(1, CHROM_LENGTH-1)
        c1 = p1[:point] + p2[point:]
        c2 = p2[:point] + p1[point:]
        return c1, c2
    return p1, p2

# Mutation (bit flip)
def mutate(chrom):
    chrom_list = list(chrom)
    for i in range(CHROM_LENGTH):
        if random.random() < MUT_RATE:
            chrom_list[i] = '1' if chrom_list[i] == '0' else '0'
    return ''.join(chrom_list)

# --- Genetic Algorithm main ---
def genetic_algorithm():
    # Initial population (your teacher's values: 12, 23, 5, 19)
    population = [encode(x) for x in [12, 23, 5, 19]]
    print("Initial Population:", population, [decode(c) for c in population])

    for gen in range(1, 6):  # run for 5 generations
        # Decode & evaluate
        decoded = [decode(c) for c in population]
        fitnesses = [fitness(x) for x in decoded]

        # Print population status
        total_fit = sum(fitnesses)
        probs = [f/total_fit for f in fitnesses]
        expected = [p*POP_SIZE for p in probs]

        print(f"\nGeneration {gen}")
        for i in range(POP_SIZE):
            print(f"x={decoded[i]}, bin={population[i]}, fit={fitnesses[i]}, "
                  f"prob={probs[i]:.3f}, exp_count={expected[i]:.2f}")

        # --- Selection (Roulette) ---
        new_pop = []
        while len(new_pop) < POP_SIZE:
            p1 = roulette_selection(population, fitnesses)
            p2 = roulette_selection(population, fitnesses)
            c1, c2 = crossover(p1, p2)
            c1, c2 = mutate(c1), mutate(c2)
            new_pop.extend([c1, c2])

        population = new_pop[:POP_SIZE]

    # Final best solution
    decoded = [decode(c) for c in population]
    fitnesses = [fitness(x) for x in decoded]
    best_idx = fitnesses.index(max(fitnesses))
    print("\nFinal Best Solution:", decoded[best_idx], population[best_idx], "fitness=", fitnesses[best_idx])

# Run it
genetic_algorithm()

Initial Population: ['01100', '10111', '00101', '10011'] [12, 23, 5, 19]

Generation 1
x=12, bin=01100, fit=144, prob=0.136, exp_count=0.54
x=23, bin=10111, fit=529, prob=0.500, exp_count=2.00
x=5, bin=00101, fit=25, prob=0.024, exp_count=0.09
x=19, bin=10011, fit=361, prob=0.341, exp_count=1.36

Generation 2
x=23, bin=10111, fit=529, prob=0.455, exp_count=1.82
x=3, bin=00011, fit=9, prob=0.008, exp_count=0.03
x=15, bin=01111, fit=225, prob=0.193, exp_count=0.77
x=20, bin=10100, fit=400, prob=0.344, exp_count=1.38

Generation 3
x=31, bin=11111, fit=961, prob=0.705, exp_count=2.82
x=5, bin=00101, fit=25, prob=0.018, exp_count=0.07
x=19, bin=10011, fit=361, prob=0.265, exp_count=1.06
x=4, bin=00100, fit=16, prob=0.012, exp_count=0.05

Generation 4
x=27, bin=11011, fit=729, prob=0.307, exp_count=1.23
x=19, bin=10011, fit=361, prob=0.152, exp_count=0.61
x=18, bin=10010, fit=324, prob=0.136, exp_count=0.55
x=31, bin=11111, fit=961, prob=0.405, exp_count=1.62

Generation 5
x=23, bin=10111, f

In [ ]:
import numpy as np
import random

mean_returns = np.array([0.12, 0.18, 0.15, 0.10])
cov_matrix = np.array([
    [0.10, 0.02, 0.04, 0.01],
    [0.02, 0.08, 0.02, 0.00],
    [0.04, 0.02, 0.09, 0.01],
    [0.01, 0.00, 0.01, 0.05]
])

risk_free_rate = 0.03

# Fitness: Sharpe ratio = (return - risk_free) / risk
def fitness(weights):
    weights = np.array(weights)
    port_return = np.dot(weights, mean_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    if port_vol == 0:
        return -999  # invalid
    return (port_return - risk_free_rate) / port_vol

# Create random portfolio (weights sum to 1)
def random_portfolio(n_assets=4):
    w = np.random.rand(n_assets)
    return w / np.sum(w)

# Selection (tournament)
def selection(pop, fitnesses):
    i, j = random.sample(range(len(pop)), 2)
    return pop[i] if fitnesses[i] > fitnesses[j] else pop[j]

# Crossover
def crossover(p1, p2):
    alpha = random.random()
    child = alpha * np.array(p1) + (1 - alpha) * np.array(p2)
    return child / np.sum(child)

# Mutation
def mutate(weights, rate=0.1):
    if random.random() < rate:
        i, j = random.sample(range(len(weights)), 2)
        weights[i] += np.random.normal(0, 0.05)
        weights[j] -= np.random.normal(0, 0.05)
        weights = np.abs(weights)
        weights /= np.sum(weights)
    return weights

# Genetic Algorithm
def genetic_portfolio(generations=100, pop_size=30, n_assets=4):
    population = [random_portfolio(n_assets) for _ in range(pop_size)]

    for gen in range(generations):
        fitnesses = [fitness(ind) for ind in population]
        best_idx = np.argmax(fitnesses)
        print(f"Gen {gen+1}: Best Sharpe = {fitnesses[best_idx]:.4f}, Weights = {population[best_idx]}")

        new_pop = []
        for _ in range(pop_size):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_pop.append(child)

        population = new_pop

    best_idx = np.argmax([fitness(ind) for ind in population])
    return population[best_idx]

# Run GA
best_portfolio = genetic_portfolio()
print("\nOptimized Portfolio Weights:", best_portfolio)
print("Expected Return:", np.dot(best_portfolio, mean_returns))
print("Volatility:", np.sqrt(np.dot(best_portfolio.T, np.dot(cov_matrix, best_portfolio))))
print("Sharpe Ratio:", fitness(best_portfolio))
